#Preparación de datos

In [16]:
# Cargar los paquetes y librerías necesarios 

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer

import warnings
warnings.filterwarnings("ignore")

In [17]:
# Cargar los datos desde el archivo CSV en el dataframe llamado df.
df = pd.read_csv("/Users/Lalis/Desktop/Proyecto analitica 2/base_proyecto.csv", sep=",", header = 0,decimal=".")  
df.head() # show the "head" -- first 5 rows of the data; note, these are rows 0...4

,DemoReqPg_CallClicks_evt_count,air_purifier_page_top,bounces,checkout_page_top,client_id,contactus_top,converted_in_7days,country,customer_service_amc_login_top,customer_service_request_login_top,...,visited_customer_service_amc_login,visited_customer_service_request_login,visited_demo_page,visited_offer_page,visited_security_solutions_page,visited_storelocator,visited_successbookdemo,visited_vacuum_cleaner_page,visited_water_purifier_page,water_purifier_page_top
0,0,0,0,0,1000148488.154624,0,0,i,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1000446930.154546,0,0,d,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1000984214.15461,0,0,i,0,0,...,0,0,0,0,0,0,0,0,1,142
3,0,0,1,0,1001576926.154625,0,0,d,0,0,...,0,0,0,1,0,0,0,0,1,107
4,0,0,1,0,1002130272.15414,0,0,d,0,0,...,0,0,0,0,0,0,0,0,0,0


Como primer paso de limpieza se eliminaron los datos que presentaban incongruencias, como el caso donde la variable objetivo tenia valores de 2 o 3, al ser tan pocos se entiende que esto pudo ser por error humano a la hora de diligenciar, adicionalmente se observo la falta de diferentes datos en los mismos casos donde la variable objetivo tenia valores incongruentes.  

In [18]:
df.info() # Para chequear qué tipo de datos tenemos 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709244 entries, 0 to 709243
Data columns (total 45 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   DemoReqPg_CallClicks_evt_count          709244 non-null  int64 
 1   air_purifier_page_top                   709244 non-null  int64 
 2   bounces                                 709244 non-null  int64 
 3   checkout_page_top                       709244 non-null  int64 
 4   client_id                               709244 non-null  object
 5   contactus_top                           709244 non-null  int64 
 6   converted_in_7days                      709244 non-null  int64 
 7   country                                 709244 non-null  object
 8   customer_service_amc_login_top          709244 non-null  int64 
 9   customer_service_request_login_top      709244 non-null  int64 
 10  date                                    709244 non-null 

In [19]:
# Limpieza de los datos -- conversión de tipos de datos que deberían ser categóricos

df['country'] = df['country'].astype('category')
df['date'] = df['date'].astype('category')
df['device'] = df['device'].astype('category')
df['region'] = df['region'].astype('category')
df['sourceMedium'] = df['sourceMedium'].astype('category')

df = df.drop(columns = 'client_id')

In [20]:
# fijación de variables categóricas
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(df.select_dtypes(exclude=['int64','float64']))
df[df.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(df.select_dtypes(exclude=['int64','float64']))
           
# fijación de variables numéricas 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(df.select_dtypes(include=['int64','float64']))
df[df.select_dtypes(include=['int64','float64']).columns] = imputer.transform(df.select_dtypes(include=['int64','float64']))

In [21]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709244 entries, 0 to 709243
Data columns (total 44 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   DemoReqPg_CallClicks_evt_count          709244 non-null  float64
 1   air_purifier_page_top                   709244 non-null  float64
 2   bounces                                 709244 non-null  float64
 3   checkout_page_top                       709244 non-null  float64
 4   contactus_top                           709244 non-null  float64
 5   converted_in_7days                      709244 non-null  float64
 6   country                                 709244 non-null  object 
 7   customer_service_amc_login_top          709244 non-null  float64
 8   customer_service_request_login_top      709244 non-null  float64
 9   date                                    709244 non-null  object 
 10  demo_page_top                           7092

In [22]:
# Limpieza de datos -- creación de dummies para variables no numéricas ("one hot encoding")

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)

pd.options.display.max_columns = None # remove the limit on the number of columns by default only 20 are shows

df.head()  # nuestro dataset tiene ahora 241 columnas (!)

,DemoReqPg_CallClicks_evt_count,air_purifier_page_top,bounces,checkout_page_top,contactus_top,converted_in_7days,customer_service_amc_login_top,customer_service_request_login_top,demo_page_top,dsls,fired_DemoReqPg_CallClicks_evt,fired_help_me_buy_evt,fired_phone_clicks_evt,goal4Completions,help_me_buy_evt_count,newUser,offer_page_top,pageviews,paid,phone_clicks_evt_count,security_solutions_page_top,sessionDuration,sessions,storelocator_top,successbookdemo_top,vacuum_cleaner_page_top,visited_air_purifier_page,visited_checkout_page,visited_contactus,visited_customer_service_amc_login,visited_customer_service_request_login,visited_demo_page,visited_offer_page,visited_security_solutions_page,visited_storelocator,visited_successbookdemo,visited_vacuum_cleaner_page,visited_water_purifier_page,water_purifier_page_top,country_i,date_2019-01-02,date_2019-01-03,date_2019-01-04,date_2019-01-05,date_2019-01-06,date_2019-01-07,date_2019-01-08,date_2019-01-09,date_2019-01-10,date_2019-01-11,date_2019-01-12,date_2019-01-13,date_2019-01-14,date_2019-01-15,date_2019-01-16,date_2019-01-17,date_2019-01-18,date_2019-01-19,date_2019-01-20,date_2019-01-21,date_2019-01-22,date_2019-01-23,date_2019-01-24,date_2019-01-25,date_2019-01-26,date_2019-01-27,date_2019-01-28,date_2019-01-29,date_2019-01-30,date_2019-01-31,date_2019-02-01,date_2019-02-02,date_2019-02-03,date_2019-02-04,date_2019-02-05,date_2019-02-06,date_2019-02-07,date_2019-02-08,date_2019-02-09,date_2019-02-10,date_2019-02-11,date_2019-02-12,date_2019-02-13,date_2019-02-14,date_2019-02-15,date_2019-02-16,date_2019-02-17,date_2019-02-18,date_2019-02-19,date_2019-02-20,date_2019-02-21,date_2019-02-22,date_2019-02-23,date_2019-02-24,date_2019-02-25,date_2019-02-26,date_2019-02-27,date_2019-02-28,date_2019-03-01,device_mobile,device_tablet,region_(not set),region_Abu Dhabi,region_Addis Ababa,region_Ajman,region_Akershus,region_Al Ahmadi Governorate,region_Al Asimah Governate,region_Al Batinah North Governorate,region_Al Farwaniyah Governorate,region_Al Jahra Governorate,region_Al Khor,region_Al Madinah Province,region_Al Qassim,region_Alabama,region_Alaska,region_Alberta,region_Algiers Province,region_Almaty Province,region_Amman Governorate,region_Andalusia,region_Andaman and Nicobar Islands,region_Andhra Pradesh,region_Arizona,region_Arkansas,region_Aseer Province,region_Assam,region_Aswan Governorate,region_Attica,region_Auckland,region_Australian Capital Territory,region_Auvergne-Rhone-Alpes,region_Baden-Wurttemberg,region_Baghdad Governorate,region_Balearic Islands,region_Bali,region_Banaadir,region_Bangkok,region_Barisal Division,region_Basel City,region_Basra Governorate,region_Bavaria,region_Beijing,region_Berlin,region_Bicol,region_Bihar,region_Binh Thuan Province,region_Braga,region_Brandenburg,region_British Columbia,region_Brunei-Muara District,region_Brussels,region_Bucharest,region_Budapest,region_Buenos Aires,region_Buskerud,region_Caicos Islands,region_Cairo Governorate,region_Calabarzon,region_California,region_Canton of Bern,region_Canton of Neuchatel,region_Capital Governorate,region_Capital Municipality,region_Capital Region of Denmark,region_Catalonia,region_Center District,region_Central Denmark Region,region_Central Development Region,region_Central Division,region_Central Governorate,region_Central Java,region_Central Region,region_Central Visayas,region_Centre,region_Chandigarh,region_Chhattisgarh,region_Chiba Prefecture,region_Chittagong Division,region_Chon Buri,region_Chuy Province,region_City of Zagreb,region_Cluj County,region_Colorado,region_Conakry,region_Connecticut,region_Cordillera Administrative Region,region_County Cork,region_County Dublin,region_County Galway,region_County Kerry,region_County Kildare,region_County Laois,region_County Limerick,region_Da Nang,region_Dadra and Nagar Haveli,region_Dakahlia Governorate,region_Dakar Region,region_Dalarna County,region_Davao Region,region_Delaware,region_Delhi,region_Dhaka Division,region_District of Columbia,re

In [23]:
y = df['converted_in_7days']
X = df.drop(columns = 'converted_in_7days')

In [24]:
# Definimos la semilla para el generador de número aleatorios
np.random.seed(77300)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, stratify=y)
# IMPORTANTE: Las muestras están estratificadas, i.e., la proporción de clientes retenidos y no-retenidos es la misma en ambos

# Chequeemos los resultados
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(567395, 754)
(567395,)
(141849, 754)
(141849,)


In [25]:
# Primero, definimos un conjunto de funciones para calcular las métricas del modelo

# Curva ROC
def plot_roc(y_test, y_pred):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1, drop_intermediate = False)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.001, 1.001])
    plt.ylim([-0.001, 1.001])
    plt.xlabel('1-Specificity (False Negative Rate)')
    plt.ylabel('Sensitivity (True Positive Rate)')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

# Matriz de Confusión: cm[0,0], cm[0,1], cm[1,0], cm[1,1]: tn, fp, fn, tp

# Sensitivity
def custom_sensitivity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fn))

# Specificity
def custom_specificity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fp))

# Positive Predictive Value
def custom_ppv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fp))

# Negative Predictive Value
def custom_npv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fn))

# Accuracy
def custom_accuracy_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return ((tn+tp)/(tn+tp+fn+fp))

In [ ]:
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

svm_estimators = []
svm_estimators.append(('normalize', MinMaxScaler())) # escalamos los datos
svm_estimators.append(('svm', svm.SVC(probability=True))) # definimos SVM con probabilidades 
     
# Definimos el modelo SVM y lo llamamos classifier_SVM
Classifier_SVM = Pipeline(svm_estimators, verbose=False)

# Entrenamos el modelo classifier_SVM sobre los datos de entrenamiento
Classifier_SVM.fit(X_train, y_train)

In [ ]:
# Usamos el modelo desarrollado, para predecir sobre los datos de prueba 
y_pred_prob = Classifier_SVM.predict_proba(X_test)[:,1] # probabilidades
class_threshold = 0.6073
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación
y_pred_prob

In [ ]:
### REFERENCIAS SOBRE RFE (Recursive Feature Elimination)
### https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15
### https://www.kite.com/python/docs/sklearn.feature_selection.RFE

from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE

# create the RFE model for the svm classifier and select attributes
rfeSVM = RFE(estimator=Classifier_SVM, n_features_to_select=20, step=1) 
rfeSVM.fit(X_train, y_train)

ranking = rfeSVM.ranking_.reshape(len(X_train.columns))

# Cuáles son las 20 variables que quedan en el modelo?
pd.DataFrame([X_test.columns,ranking]).transpose().sort_values(1).head(30)